# Mapping with plotly express
plotly.express

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import pandas as pd
import plotly.offline as pyo
import plotly.express as px
import json

from datetime import datetime, timedelta
offset_hrs=-11

def get_secret():
    secret_name = "202012-awsmap"
    region_name = "us-west-2"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager',   region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    secret_value_json_object = json.loads(get_secret_value_response['SecretString'])
    return secret_value_json_object

def get_utc_string(from_tz):
    from_tz_utc = datetime.strptime(from_tz, "%Y-%m-%d %H:%M") + timedelta(hours=offset_hrs) 
    ret_utc = from_tz_utc.strftime("%Y-%m-%dT%H:%M:%S")
    return ret_utc

def get_local_string(from_utc):
    from_tz_utc = datetime.strptime(from_utc[:-4], "%Y-%m-%dT%H:%M:%S.%f") - timedelta(hours=offset_hrs)
    ret_local = from_tz_utc.strftime("%Y/%m/%d %H:%M")
    return ret_local

def run_query(group_text, from_ts, to_ts):
    time_from=get_utc_string(from_ts)
    time_to=get_utc_string(to_ts)

    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('saubury-001')

    fe = Key('time').between(time_from, time_to);
    response = table.scan(FilterExpression=fe)
    items = response['Items']

    
    for item in items:
        lat_array.append(item['Latitude'])
        lon_array.append(item['Longitude'])
        vistime_array.append(get_local_string(item['time']))
        group_array.append(group_text)


lat_array = []
lon_array = []
vistime_array = []
group_array = []
secret_value_json_object = get_secret()

run_query('Monday Morning', '2021-01-04 08:10', '2021-01-04 12:30')
run_query('Monday Afternoon', '2021-01-04 13:00', '2021-01-04 15:30')
run_query('Monday Night', '2021-01-04 15:30', '2021-01-04 19:30')
run_query('Tuesday Morning', '2021-01-05 08:05', '2021-01-05 12:35')
run_query('Tuesday Evening', '2021-01-05 15:55', '2021-01-05 18:18')

mydict = {'Lat': lat_array, 'Lon': lon_array, 'Time' : vistime_array, 'Adventure' : group_array}
mydf = pd.DataFrame(data=mydict)
#print(mydf)

pyo.init_notebook_mode()
fig = px.line_mapbox(mydf, lat="Lat", lon="Lon", color="Adventure", hover_name="Time", zoom=100, height=0)
fig.update_layout( mapbox_zoom=16, mapbox_center_lat = -33.805,
    margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox_style="streets", mapbox_accesstoken=secret_value_json_object['mapbox_access_token'])

fig.show()